In [1]:
from scipy.io import loadmat
from numpy import zeros as np_zeros, ones as np_ones
from numpy import vectorize as np_vectorize

from lib.stft import stft
from lib.non_iter_ls_inv_stft import non_iter_ls_inv_stft


# Test variables
num_rows, num_elements, num_beams = 1624, 65, 128
len_each_section = 16;
frac_overlap = 0.9;
padding = 16;
# winInfo = {@rectwin};
CHANDAT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/chandat.mat'
chandat = loadmat(CHANDAT_FNAME)['chandat']

NEW_STFT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/new_stft.mat'
new_stft_f = loadmat(NEW_STFT_FNAME)
new_stft_real = new_stft_f['new_stft_real']
new_stft_imag = new_stft_f['new_stft_imag']


In [2]:
chandat_stft = stft(chandat, len_each_section, frac_overlap, padding)
chandat_stft['origSigSize'] = [num_rows, num_elements, num_beams]


In [3]:
stft_data = chandat_stft['stft']

In [4]:
from numpy.linalg import norm
norm(stft_data)

1810982.5125408696

In [5]:
new_stft = new_stft_real + 1j*new_stft_imag


In [6]:
# create new and old stfts
from copy import copy

chandat_stft_new = copy(chandat_stft)
chandat_stft_new['stft'] = new_stft


In [7]:
chandat_new = non_iter_ls_inv_stft(chandat_stft_new)


norm(DlsArr) = 320.4746479832687
norm(invFT) = 167013460.0
norm(yEst) = 281768794.51286405
norm(sigOut) = 35911183.26331446
